# MLOps Sentiment Analysis - Sistema di Monitoraggio Completo

Questo notebook implementa il sistema completo di monitoraggio e retraining per il modello di sentiment analysis. Include:
- Download del dataset
- Valutazione del modello
- Logging delle predizioni
- Tracking delle metriche
- Rilevazione del drift
- Trigger di retraining
- Visualizzazioni dei risultati

In [ ]:
# Download del dataset
from datasets import load_dataset
from pathlib import Path

# Carica e salva il dataset
data_dir = Path("data/raw")
data_dir.mkdir(parents=True, exist_ok=True)

dataset = load_dataset("tweet_eval", "sentiment")
dataset.save_to_disk(str(data_dir / "tweet_eval"))

print(f"Dataset scaricato e salvato in {data_dir / 'tweet_eval'}")
print(f"Train samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")

Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 433721.52 examples/s]


In [ ]:
# Esempio di inferenza di base
from src.sentiment_model import analyze_sentiment

text = "Oggi è una bella giornata!"
result = analyze_sentiment(text)
print(f"Testo: {text}")
print(f"Risultato: {result}")

{'Negativo': 0.054527703672647476, 'Neutro': 0.7331935167312622, 'Positivo': 0.21227875351905823}


## Valutazione del Modello su Test Set

Valutiamo il modello su un campione del test set calcolando accuracy e F1 score.

In [ ]:
# Valutazione del modello
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from datasets import load_dataset
from src.sentiment_model import analyze_sentiment
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Carica il test set
dataset = load_dataset("tweet_eval", "sentiment")
test_dataset = dataset['test']

# Usa i primi 50 campioni per la valutazione (per velocità)
sample_size = min(50, len(test_dataset))
sample_indices = np.random.choice(len(test_dataset), sample_size, replace=False)

# Mapping delle label
label_mapping = {0: "Negativo", 1: "Neutro", 2: "Positivo"}
reverse_mapping = {"Negativo": 0, "Neutro": 1, "Positivo": 2}

# Valutazione
predictions = []
true_labels = []
confidences = []

for idx in sample_indices:
    sample = test_dataset[int(idx)]
    text = sample['text']
    true_label = label_mapping[sample['label']]
    
    # Effettua la predizione
    result = analyze_sentiment(text)
    pred_label = max(result, key=result.get)
    confidence = result[pred_label]
    
    predictions.append(pred_label)
    true_labels.append(true_label)
    confidences.append(confidence)

# Calcola le metriche
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted', zero_division=0)

print("=" * 60)
print("VALUTAZIONE DEL MODELLO SUL TEST SET")
print("=" * 60)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")
print(f"Confidenza media: {np.mean(confidences):.4f}")
print(f"\n{classification_report(true_labels, predictions, zero_division=0)}")